In [1]:
from web3 import Web3
from eth_account import Account

#此代码默认已授权合约调用足够代币
RPC = 'https://arb-mainnet.g.alchemy.com/v2/qL7ObTkY8Vn1wJRwIjjzUAd-BFLSRO8o'
P_KEY = input('Enter your private key: ')

w3= Web3(Web3.HTTPProvider(RPC))
print("input address: ", Account.from_key(P_KEY).address)

input address:  0x92C88EB0F89465aC3D9453F84E20DD1b2015ffF2


In [3]:
from eth_abi import encode

class TxHelper:
    
    eth_address = Account.from_key(P_KEY).address
    # get nonce
    @staticmethod
    def nonce():
        return w3.eth.get_transaction_count(TxHelper.eth_address)
    # get txn params
    @staticmethod
    def build_txn_params( gas_limit=2000000):
        return {
            'from': TxHelper.eth_address,
            'gas': gas_limit,
            'gasPrice': w3.eth.gas_price,
            'nonce': TxHelper.nonce(),
        }
    
    # build transaction, signed and then send
    @staticmethod
    def send(transaction_data):
        signed_txn = Account.sign_transaction(transaction_data, P_KEY)
        txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
        return txn_hash
    
    @staticmethod
    def call_contract_function(contract_address,function_selector ,params:list,params_type:list):
        # 构建函数调用的数据
        data = function_selector + str(encode(params_type, params).hex())
        transaction_data = {
            'to': contract_address,
            'data': data,
        }
        transaction_data.update(TxHelper.build_txn_params())
        signed_txn = Account.sign_transaction(transaction_data, P_KEY)
        txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
        return txn_hash
    
    @staticmethod
    def wait_for_complete(txn_hash):
        receipt = w3.eth.wait_for_transaction_receipt(txn_hash)
        if receipt is None or receipt['status'] != 1:
            raise Exception('transaction failed')
        return receipt

In [4]:
import json
class Xmgn:
    #stake withdraw
    def __init__(self):
        with open('./abi/xmgn.abi') as f:
            xmgn_abi = json.load(f)
        
        with open('./abi/erc20.abi') as f:
            erc20_abi = json.load(f)
        self.contract = w3.eth.contract(address="0x25B9f82D1F1549F97b86bd0873738E30f23D15ea", abi=xmgn_abi)
        self.mgn_contract = w3.eth.contract(address="0xFc77b86F3ADe71793E1EEc1E7944DB074922856e", abi=erc20_abi)
    def balanceOf(self, address):
        return self.contract.functions.balanceOf(address).call()
    def mgn_balanceOf(self, address):
        return self.mgn_contract.functions.balanceOf(address).call()
    def withdraw(self,amount):
        transaction_data = self.contract.functions.withdraw(amount).build_transaction(TxHelper.build_txn_params())
        return TxHelper.send(transaction_data)
    def stake(self,amount):
        transaction_data = self.contract.functions.stake(amount).build_transaction(TxHelper.build_txn_params())
        return TxHelper.send(transaction_data)

In [5]:
from eth_abi import encode
class StMgn:
    #deposit redeeme
    def __init__(self):
        self.contract_address='0x5A45fa1EB7033B488f160Fe8C852eF867547fF65'

        self.deposit_function_selector = '0x6e553f65'
        self.redeem_function_selector = '0xba087652'

        with open('./abi/erc20.abi') as f:
            erc20_abi = json.load(f)
        self.erc20  = w3.eth.contract(address=self.contract_address, abi=erc20_abi)
    
    
    def balanceOf(self, address):
        return self.erc20.functions.balanceOf(address).call()

    #Function: deposit(uint256 amount, address to) ***
    def deposit(self,amount,to):
        params_type = ['uint256','address']
        params = [amount,to]
        return TxHelper.call_contract_function(self.contract_address,self.deposit_function_selector,params,params_type)
    
    # Function: redeem(uint256 _shares, address _receiver, address _owner) ***
    def redeem(self,amount,to,owner):
        params_type = ['uint256','address','address']
        params = [amount,to,owner]
        return TxHelper.call_contract_function(self.contract_address,self.redeem_function_selector,params,params_type)

In [ ]:
import time
xmgn=Xmgn()
stmgn = StMgn()

# withdraw -> deposit-> redeem ->stake
# xMGN-MGN-stMGN v2-MGN-xMGN
#1. get xmgn balance 
xmgn_balance = xmgn.balanceOf(TxHelper.eth_address)
print("address withdraw xmgn balance %f" %(xmgn_balance/10**18))
#2. withdraw xmgn to mgn 赎回xmgn
hash = xmgn.withdraw(xmgn_balance)
print("withdrew hash "+hash.hex())
TxHelper.wait_for_complete(hash)
#3. get mgn balance
mgn_balance = xmgn.mgn_balanceOf(TxHelper.eth_address)
open_amt = mgn_balance
print("address mgn balance %f" %(mgn_balance/10**18))
#4. deposit mgn to stMGN 存至自动复利器
hash = stmgn.deposit(mgn_balance,TxHelper.eth_address)
TxHelper.wait_for_complete(hash)
print("deposit hash "+hash.hex())



In [9]:
#5. get stMGN balance
stmgn_amt= stmgn.balanceOf(TxHelper.eth_address)
print("address redeem stmgn balance %f" %(stmgn_amt/10**18))

time.sleep(150)

#6. redeem stMGN to mgn 取自自动复利器
hash = stmgn.redeem(stmgn_amt,TxHelper.eth_address,TxHelper.eth_address)
TxHelper.wait_for_complete(hash)
print("redeem hash "+hash.hex())
#7. get mgn balance
mgn_balance = xmgn.mgn_balanceOf(TxHelper.eth_address)
close_amt = mgn_balance
reward = close_amt - open_amt
print("address mgn balance %f" %(mgn_balance/10**18))
#8. stake mgn to xmgn
hash = xmgn.stake(mgn_balance)
TxHelper.wait_for_complete(hash)
print("stake hash "+hash.hex())
print('excute reward '+str(reward))
print('cumulative reward')

address redeem stmgn balance 401.601262
redeem hash 0x67e471ac599deb891e71d0f02782ac72f361b455a9adca5affffe35fcdb1e0c2
address mgn balance 403.514793


Exception: transaction failed

In [13]:
import time
time.sleep(5)
print(4)

4
